# Языковые модели

Какое слово в последовательности вероятнее: 

Поезд прибыл на
* вокзал
* север

Какая последовательность вероятнее:
* Вокзал прибыл поезд на
* Поезд прибыл на вокзал

Языковая модель [language model, LM]  позволяет оценить вероятность следующего слова в последовательности  $P(w_n | w_1, \ldots, w_{n-1})$ и оценить вероятность всей последовательности слов $P(w_1, \ldots, w_n)$.

### Приложения:
* Задачи, в которых нужно обработать сложный и зашумленный вход: распознавание речи, распознавание сканированных и рукописных текстов;
* Исправление опечаток
* Машинный перевод
* Подсказка при наборе 

### Виды моделей:
* Счетные модели
    * цепи Маркова
* Нейронные модели, обычно реккурентные нейронные сети с LSTM/GRU
* seq2seq архитектуры

## Модель $n$-грам

Пусть $w_{1:n}=w_1,\ldots,w_m$ – последовательность слов.

Цепное правило:
$ P(w_{1:m}) = P(w_1) P(w_2 | w_1) P(w_3 | w_{1:2}) \ldots P(w_m | w_{1:m-1}) = \prod_{k=1}^{m} P(w_k | w_{1:k-1}) $

Но оценить $P(w_k | w_{1:k-1})$ не легче! 

Переходим к $n$-грамам: $P(w_{i+1} | w_{1:i}) \approx P(w_{i+1} | w_{i-n:i})  $ , то есть, учитываем $n-1$ предыдущее слово. 

Модель
* униграм: $P(w_k)$
* биграм: $P(w_k | w_{k-1})$
* триграм: $P(w_k | w_{k-1} w_{k-2})$


Т.е. используем Марковские допущения о длине запоминаемой цепочки.

* Вероятность следующего слова в последовательности: $ P(w_{i+1} | w_{1:i}) \approx P(w_{i-n:i}) $
* Вероятность всей последовательности слов: $P(w_{1:n}) = \prod_{k=1}^l P(w_k | w_{k-n+1: k-1}) $

### Качество модели  $n$-грам

Перплексия: насколько хорошо модель предсказывает выборку. Чем ниже значение перплексии, тем лучше.

$PP(\texttt{LM}) = 2 ^ {-\frac{1}{m} \log_2 \texttt{LM} (w_i | w_{1:i-1})}$

## Счетные языковые модели

### ММП оценки вероятностей
$ P_{MLE}(w_k | w_{k-n+1:k-1}) = \frac{\texttt{count}(w_{k-n+1:k-1} w_k )}{\texttt{count}(w_{k-n+1:k-1} )} $

В модели биграм:

$ P_{MLE}(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k )}{\texttt{count}(w_{k-1} )} $

Возникает проблема нулевых вероятностей!

### Аддитивное сглаживание Лапласа

$ P(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k ) + \alpha}{\texttt{count}(w_{k-1} ) + \alpha |V|} $

![AiB](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/aib.png)  

BOS А и Б сидели на трубе EOS

BOS А упало Б пропало EOS

BOS что осталось на трубе EOS




$P($ и $| $ A $) = \frac{1}{2}$

$P($ Б $| $ и $) = \frac{1}{1}$

$P($ трубе $| $ на $) = \frac{2}{2}$

$P($ сидели $| $ Б $) = \frac{1}{2}$

$P($ на $| $ сидели $) = \frac{1}{2}$


## Модели биграм в NLTK

In [ ]:
import nltk

from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout

In [ ]:
names = [name.strip().lower() for name in open('dinos.txt').readlines()]
print(names[:10])

In [ ]:
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)

print(list(freq.keys()))

In [ ]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))
cfreq['a']

In [ ]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

In [ ]:
from math import log
log(cprob['a'].prob('a')) + log(cprob['a'].prob('b')) + log(cprob['a'].prob('c'))

In [ ]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l
print('p(a) = %1.4f' %unigram_prob('a'))

In [ ]:
[bi for bi in nltk.bigrams('aachenosaurus')]

#### Задание 1

1. Напишите функцию для оценки вероятности имени динозавра. 
2. Найдите наиболее вероятное имя динозавра из данного списка. 

In [ ]:
cprob["a"].generate()

#### Задание 2

Напишите функцию для генерации нового имени динозавра фиксированной длины.

## Нейронные языковые модели

* Вход: $n$-грамы $w_{1:k}$
* $v(w_i)$ – эмбеддинг слова $w_i$, $v(w_i) \in \mathbb{R}^{d_{emb}}$, $d_{emb}$ – размерность эмбеддинга, $v(w) = E_{[w]}$
* $x = [v(w_1), v(w_2), \ldots , v(w_k)]$

$\widehat{y} = P(w_i | w_{1:k} ) = \texttt{LM}(w_{1:k}) = \texttt{softmax}(hW^2 +b^2)$

$h = g(xW^1+b^1)$

$w_i \in V$, $E \in \mathbb{R}^{|V|\times d_{emb}}, W^1 \in \mathbb{R}^{k \cdot d_{emb} \times d_{hid}}, b^1 \in \mathbb{R} ^ {d_{hid}}, W^2 \in \mathbb{R}^{d_{hid} \times |V|}, b^2 \in \mathbb{R} ^ {|V|}$

![nnlm](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/nlm1.png)

### Семплирование в нейронных языковых моделях 
### (Генерация текстов с помощью нейронных языковых моделей)

1. Задать начальную последовательность символов длины $k$ (/слов)
2. Предсказать распределение вероятностей слов с условием на $k$ предыдущих слов
3. 1. Выбрать слово с наибольшей вероятностью
3. 2. Выбрать слово по предсказаному распределению
4. Сдвинуть окно на одно слово и повторить 

#### Линейный поиск  (beam search)
Всегда помним $h$ наиболее вероятных гипотез:
1. Для генерации первого слова в последоватительности генерируем $h$ кандидатов, а не 1
2. Генерируем $h \times h$ кандидатов для второго слова и храним только $h$ наиболее вероятных


![nnlm](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/nlm2.png)

In [ ]:
alphabet = list(set(chars))
print('total chars:', len(alphabet))

In [ ]:
maxlen = 5
step = 1
ngrams = []
next_chars = []
for name in names:
    for i in range(0, len(name) - maxlen, step):
        ngrams.append(' '.join([char for char in name[i: i + maxlen]]))
        next_chars.append(name[i + maxlen])
print('nb ngrams:', len(ngrams))
print(ngrams[0],next_chars[0])
print(ngrams[1],next_chars[1])

In [ ]:
tokenizer = Tokenizer(num_words=len(alphabet))
tokenizer.fit_on_texts(ngrams)

sequences = tokenizer.texts_to_sequences(ngrams)
X_train = pad_sequences(sequences, maxlen=maxlen)
sequences = tokenizer.texts_to_sequences(next_chars)
y_train = tokenizer.sequences_to_matrix(sequences)
X_train[0]

In [ ]:
y_train[0]

In [ ]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [ ]:
model = Sequential()

model.add(Embedding(len(alphabet), 50, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation = 'softmax'))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'softmax'))
model.add(Dropout(0.2))
model.add(Dense(len(alphabet), activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train,y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose = 0)

In [ ]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) #/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)), p = preds)
    return probas

In [ ]:
import random
import sys
import numpy as np

generated = ''
seed = 'katya'
generated += seed
print('----- Generating with seed: "' + seed + '"')
print(generated)

for i in range(8):
    sequences = tokenizer.texts_to_sequences([' '.join([char for char in generated[-maxlen:]])])
    X_pred = pad_sequences(sequences, maxlen=maxlen)
    preds = model.predict(X_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = index_char[next_index]
    generated += next_char
    print(generated)

#### Задание 3

Измените код выше так, чтобы генерировались панграмы – имена динозавров, не содержащие повторяющихся букв

#### Задание 4

Измените функцию семлирования `sample`: добавьте параметр `t`, изпольузуемый для шкалирования вероятностей  `preds`: ```
preds /= t
``` 

Как использование этого параметра влияет на генерируемые имена?

### Рекуррентные нейронные языковые модели

RNN позволяют уйти от Марковских допущений и позволяют учитывать предысторию произвольной длины.

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

$y_n = RNN(x_{1:n})$, $y_n \in \mathbb{R}^{d_{out}}$

Для каждого префикса $x_{i:i}$ $y_i$ – выходной вектор.

$y_i = RNN(x_{1:i})$

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

![rnn](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/rnn1.png)

$R$ –  рекурсивная функция с двумя входами: $x_i$ и $s_{i-1}$ (вектор состояния)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(s_{i-1} W^s + x_i W^x +b)$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{d_{out}}$, $s_i \in \mathbb{R}^{d_{out}}$

$W^x \in \mathbb{R}^{d_{in} \times d_{in}}$, $W^s \in \mathbb{R}^{d_{out} \times d_{out}}$

![rnn](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/rnn4.png)

#### Обучение RNN: backpropogation through time (BPTT)

![rnn](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/rnn3.png)

#### Сценарии использования RNN

* Acceptor: только $y_n$, используемый для дальнейшего предсказания / классификации

![rnn](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/rnn5.png)


* Encoder: только $y_n$

![rnn](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/rnn5.png)

* Transducer: выход $t_i$ для каждого входа $x_{1:i}$ – языковые модели,  sequence labelling

![rnn](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/rnn2.png)

#### Двунаправленная рекуррентная нейронная сеть

$x_{1:n}$ – входная последовательность

$s_i^f$ –  "прошлое / прямое состояние" – основано на $x_{1:i}$

$s_i^b$ –  "будущее / обратное состояние" – основано на $x_{n:i}$

$y_i = [O(s_i^f), O(s_i^b)] = [y_i^f, y_i^b]$

![rnn](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/birnn.png)

#### Управляемые архитектуры

RNN трудно обучать: проблема исчезающего градиента. Уйти от нее помогают управляемые нейроны специального вида: LSTM и GRU.

$s_i$ – память нейронной сети. Каждое использование $R$ считывает и видоизменяет всю память. 

Управляемый доступ к памяти: $g \in {0,1}^n$:

$s_{i+1} \leftarrow g \odot x + (1-g) \odot s_{i}$

Дифференцируемое управление:

$g \in \mathbb{R}^n $:

$s_{i+1} \leftarrow \sigma(g) \odot x + (1-g) \odot s_{i}$

http://colah.github.io/posts/2015-08-Understanding-LSTMs/
    

In [ ]:
from keras.utils import to_categorical
import numpy as np

X_names = ['bos ' + ' '.join(name) for name in names]
Y_names = [' '.join(name) + ' eos' for name in names]
maxlen = max([len(name) for name in names])+1

In [ ]:
tokenizer = Tokenizer(num_words=len(alphabet)+2)
tokenizer.fit_on_texts(X_names+Y_names)

sequences = tokenizer.texts_to_sequences(X_names)
X_train = pad_sequences(sequences, maxlen=maxlen, padding='post')


sequences = tokenizer.texts_to_sequences(Y_names)
Y_train = pad_sequences(sequences, padding='post')


Y_train_cat  = [to_categorical(sent, num_classes=len(alphabet)+2) for sent in Y_train]
Y_train =  np.asarray(Y_train_cat)

In [ ]:
print(X_names[0])
print(Y_names[0])


print(X_train.shape)
print(Y_train.shape)

print(tokenizer.word_index['bos'])
print(tokenizer.word_index['eos'])

In [ ]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [ ]:
model = Sequential()

model.add(Embedding(len(alphabet)+2, 30, input_length=maxlen))
model.add(LSTM(128, return_sequences = True))

model.add(Dense(len(alphabet)+2, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
for iteration in range(1, 20):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, Y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose = 1)

In [ ]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) #/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)+2), p = preds)
    return probas

In [ ]:
generated = ''
seed = 'bos'
generated += seed + ' '
print('----- Generating with seed: "' + seed + '"')
print(generated)


for i in range(7): 
    sequences = tokenizer.texts_to_sequences([seed])
    X_pred = pad_sequences(sequences, maxlen=maxlen, padding = 'post')

    preds = model.predict(X_pred, verbose=0)[0]
    samples = [sample(p) for p in preds]
    next_index = samples[i]
    while next_index == 0 or next_index == 10:
        samples = [sample(p) for p in preds]
        next_index = samples[i]
    next_char = index_char[next_index]
    generated += next_char + ' '
    print(generated)
    seed += next_char
    if next_char == 'eos':
        break
    

#### Задание 5

Измените функцию семлирования `sample`: сейчас в ней используется простой выбор наиболее вероятного следующего элемента. Замените его на лучевой поиск [beam search].

Как использование этой функции семплирования влияет на генерируемые имена?